<a href="https://colab.research.google.com/github/svangala3/Driver_drowsiness/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving MRL_dataset.zip to MRL_dataset (1).zip


In [3]:
for fn in uploaded.keys():
  print('user uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

user uploaded file "MRL_dataset (1).zip" with length 344824623 bytes


In [4]:
from zipfile import ZipFile

# Access the uploaded file by its name
file_name = list(uploaded.keys())[0]  # Get the name of the uploaded file

with ZipFile(file_name, 'r') as zip:
    zip.extractall()
    print("Done")

Done


In [5]:
import cv2
import numpy as np
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Define dataset paths
dataset_path = "/content/data"

# Subfolders
subfolders = ["train", "test", "val"]
categories = {"awake": 1, "sleepy": 0}  # Mapping: awake -> open (1), sleepy -> closed (0)

data = []
labels = []

# Load images from train, test, and val folders
for subfolder in subfolders:
    subfolder_path = os.path.join(dataset_path, subfolder)

    for category in categories:
        class_label = categories[category]
        category_path = os.path.join(subfolder_path, category)

        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)

            try:
                image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
                image = cv2.resize(image, (24, 24))  # Resize to 24x24 pixels
                data.append(image)
                labels.append(class_label)
            except Exception as e:
                print(f"Error loading image {img_name}: {e}")

# Convert to numpy arrays and normalize
data = np.array(data).reshape(-1, 24, 24, 1) / 255.0
labels = to_categorical(np.array(labels), num_classes=2)

# Split dataset into training and test sets (if needed)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print(f"Dataset loaded: {len(X_train)} training images, {len(X_test)} testing images")


Dataset loaded: 67918 training images, 16980 testing images


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(24, 24, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Reduces overfitting
    Dense(2, activation='softmax')  # Output: 2 classes (awake, sleepy)
])

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

# Save the trained model
model.save("/content/eye_drowsiness_model.h5")

print("Model training complete and saved!")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 57s 26ms/step - accuracy: 0.8872 - loss: 0.2700 - val_accuracy: 0.9688 - val_loss: 0.0926
Epoch 2/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 77s 23ms/step - accuracy: 0.9675 - loss: 0.0894 - val_accuracy: 0.9757 - val_loss: 0.0662
Epoch 3/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 51s 24ms/step - accuracy: 0.9740 - loss: 0.0718 - val_accuracy: 0.9789 - val_loss: 0.0561
Epoch 4/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 80s 23ms/step - accuracy: 0.9783 - loss: 0.0597 - val_accuracy: 0.9812 - val_loss: 0.0527
Epoch 5/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 50s 23ms/step - accuracy: 0.9812 - loss: 0.0525 - val_accuracy: 0.9800 - val_loss: 0.0558
Epoch 6/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 82s 23ms/step - accuracy: 0.9822 - loss: 0.0503 - val_accuracy: 0.9841 - val_loss: 0.0455
Epoch 7/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 82s 23ms/step - accuracy: 0.9831 - loss: 0.0472 - val_accuracy: 0.9830 - val_loss: 0.0479
Epoch 8/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 83s 24ms/step - accuracy: 0.9845 -

Model training complete and saved!


In [7]:
import time
import pygame
from tensorflow.keras.models import load_model
import cv2
import numpy as np

# Load the trained model
model = load_model("/content/eye_drowsiness_model.h5")

# Initialize alarm system
pygame.mixer.init()
pygame.mixer.music.load('/content/alarm.wav')  # Upload an alarm sound (beep)

# Load OpenCV's pre-trained face & eye detection models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Start video capture
cap = cv2.VideoCapture(0)  # Use the webcam (0 = default webcam)

eye_closed_start = None  # Track time of eye closure
alarm_on = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)

        for (ex, ey, ew, eh) in eyes:
            eye = roi_gray[ey:ey+eh, ex:ex+ew]
            eye = cv2.resize(eye, (24, 24))
            eye = eye / 255.0
            eye = eye.reshape(1, 24, 24, 1)
            prediction = model.predict(eye)

            if np.argmax(prediction) == 0:  # Eye closed (Sleepy)
                if eye_closed_start is None:
                    eye_closed_start = time.time()
                else:
                    elapsed_time = time.time() - eye_closed_start
                    if elapsed_time >= 0.25 and not alarm_on:
                        pygame.mixer.music.play(-1)  # Play alarm
                        alarm_on = True
            else:  # Eye open (Awake)
                eye_closed_start = None
                if alarm_on:
                    pygame.mixer.music.stop()  # Stop alarm
                    alarm_on = False

            break  # Process only first detected eye

    cv2.imshow('Drowsiness Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cap.release()
cv2.destroyAllWindows()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: ALSA: Couldn't open audio device: No such file or directory

In [8]:
!apt-get update
!apt-get install -y portaudio19-dev
!pip install PyAudio
!pip install pygame

import time
import pygame
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import os  # Import the os module

# Load the trained model
model = load_model("/content/eye_drowsiness_model.h5")

# Initialize alarm system
# Set the SDL_AUDIODRIVER environment variable to 'dummy' to run without audio output
os.environ['SDL_AUDIODRIVER'] = 'dummy'
pygame.mixer.init()
# Get the absolute path of the alarm file using os.path.join
alarm_path = os.path.join(os.path.dirname(__file__), '/content/alarm.wav')
# Load the alarm sound
pygame.mixer.music.load(alarm_path)

# Load OpenCV's pre-trained face & eye detection models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Start video capture
cap = cv2.VideoCapture(0)  # Use the webcam (0 = default webcam)

eye_closed_start = None  # Track time of eye closure
alarm_on = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)

        for (ex, ey, ew, eh) in eyes:
            eye = roi_gray[ey:ey+eh, ex:ex+ew]
            eye = cv2.resize(eye, (24, 24))
            eye = eye / 255.0
            eye = eye.reshape(1, 24, 24, 1)
            prediction = model.predict(eye)

            if np.argmax(prediction) == 0:  # Eye closed (Sleepy)
                if eye_closed_start is None:
                    eye_closed_start = time.time()
                else:
                    elapsed_time = time.time() - eye_closed_start
                    if elapsed_time >= 0.25 and not alarm_on:
                        # Simulate playing the alarm
                        print("Alarm triggered!")
                        alarm_on = True
            else:  # Eye open (Awake)
                eye_closed_start = None
                if alarm_on:
                    # Simulate stopping the alarm
                    print("Alarm stopped.")
                    alarm_on = False

            break  # Process only first detected eye

    cv2.imshow('Drowsiness Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cap.release()
cv2.destroyAllWindows()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,235 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,639 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,664 kB]
Hit:13 https://ppa.launchpadcontent.ne

NameError: name '__file__' is not defined

In [12]:
!apt-get update
!apt-get install -y portaudio19-dev
!pip install PyAudio
!pip install pygame

import time
import pygame
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import os  # Import the os module

# Load the trained model
model = load_model("/content/eye_drowsiness_model.h5")

# Initialize alarm system
# Set the SDL_AUDIODRIVER environment variable to 'dummy' to run without audio output
os.environ['SDL_AUDIODRIVER'] = 'dummy'
pygame.mixer.init()
# Use the direct path to the alarm file instead of relying on __file__
alarm_path = '/content/alarm.wav'
# Load the alarm sound
pygame.mixer.music.load(alarm_path)

# Load OpenCV's pre-trained face & eye detection models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Start video capture
cap = cv2.VideoCapture(0)  # Use the webcam (0 = default webcam)

eye_closed_start = None  # Track time of eye closure
alarm_on = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)

        for (ex, ey, ew, eh) in eyes:
            eye = roi_gray[ey:ey+eh, ex:ex+ew]
            eye = cv2.resize(eye, (24, 24))
            eye = eye / 255.0
            eye = eye.reshape(1, 24, 24, 1)
            prediction = model.predict(eye)

            if np.argmax(prediction) == 0:  # Eye closed (Sleepy)
                if eye_closed_start is None:
                    eye_closed_start = time.time()
                else:
                    elapsed_time = time.time() - eye_closed_start
                    if elapsed_time >= 0.25 and not alarm_on:
                        # Simulate playing the alarm
                        print("Alarm triggered!")
                        alarm_on = True
            else:  # Eye open (Awake)
                eye_closed_start = None
                if alarm_on:
                    # Simulate stopping the alarm
                    print("Alarm stopped.")
                    alarm_on = False

            break  # Process only first detected eye

    cv2.imshow('Drowsiness Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cap.release()
cv2.destroyAllWindows()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

Alternate Code try this with webcam permisson on your device

In [14]:
# Install necessary libraries in Google Colab
!pip install opencv-python
!pip install tensorflow

import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint


In [15]:
# Set up the path to your dataset
train_dir = '/content/data/train'
val_dir = '/content/data/val'
test_dir = '/content/data/test'

# Define ImageDataGenerator for image augmentation and rescaling
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories and apply the necessary transformations
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(64, 64),
                                                    batch_size=32,
                                                    class_mode='binary')

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(64, 64),
                                                  batch_size=32,
                                                  class_mode='binary')


Found 50937 images belonging to 2 classes.
Found 16980 images belonging to 2 classes.
Found 16981 images belonging to 2 classes.


In [16]:
# Define the CNN model architecture
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Add fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Dense(1, activation='sigmoid'))  # Binary output (awake vs sleepy)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model architecture
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         589,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 683,329 (2.61 MB)

 Trainable params: 683,329 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Train the model
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=val_generator,
                    callbacks=[ModelCheckpoint('best_model.h5', save_best_only=True)])


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - accuracy: 0.7368 - loss: 0.4921

1592/1592 ━━━━━━━━━━━━━━━━━━━━ 448s 280ms/step - accuracy: 0.7369 - loss: 0.4920 - val_accuracy: 0.9577 - val_loss: 0.1144
Epoch 2/10
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step - accuracy: 0.9345 - loss: 0.1735

1592/1592 ━━━━━━━━━━━━━━━━━━━━ 442s 277ms/step - accuracy: 0.9345 - loss: 0.1735 - val_accuracy: 0.9764 - val_loss: 0.0677
Epoch 3/10
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step - accuracy: 0.9544 - loss: 0.1235

1592/1592 ━━━━━━━━━━━━━━━━━━━━ 442s 278ms/step - accuracy: 0.9544 - loss: 0.1235 - val_accuracy: 0.9785 - val_loss: 0.0628
Epoch 4/10
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 440s 276ms/step - accuracy: 0.9628 - loss: 0.1079 - val_accuracy: 0.9707 - val_loss: 0.0821
Epoch 5/10
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step - accuracy: 0.9683 - loss: 0.0923

1592/1592 ━━━━━━━━━━━━━━━━━━━━ 438s 275ms/step - accuracy: 0.9683 - loss: 0.0923 - val_accuracy: 0.9826 - val_loss: 0.0529
Epoch 6/10
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 443s 278ms/step - accuracy: 0.9682 - loss: 0.0888 - val_accuracy: 0.9813 - val_loss: 0.0588
Epoch 7/10
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 439s 276ms/step - accuracy: 0.9709 - loss: 0.0818 - val_accuracy: 0.9790 - val_loss: 0.0602
Epoch 8/10
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - accuracy: 0.9716 - loss: 0.0832

1592/1592 ━━━━━━━━━━━━━━━━━━━━ 443s 278ms/step - accuracy: 0.9716 - loss: 0.0832 - val_accuracy: 0.9818 - val_loss: 0.0520
Epoch 9/10
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - accuracy: 0.9750 - loss: 0.0758

1592/1592 ━━━━━━━━━━━━━━━━━━━━ 439s 276ms/step - accuracy: 0.9750 - loss: 0.0758 - val_accuracy: 0.9856 - val_loss: 0.0437
Epoch 10/10
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - accuracy: 0.9758 - loss: 0.0705

1592/1592 ━━━━━━━━━━━━━━━━━━━━ 445s 279ms/step - accuracy: 0.9758 - loss: 0.0705 - val_accuracy: 0.9857 - val_loss: 0.0385


In [3]:
import cv2
import numpy as np
from playsound import playsound
import tensorflow as tf

# Load the pre-trained Haar Cascade Classifiers for face and eyes detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Load the trained model (replace with the actual path to your model)
model = tf.keras.models.load_model('best_model.h5')

# Path to your alarm sound file
alarm_file_path = '/content/alarm.wav'  # Replace with the correct path to your audio file

# Initialize webcam (0 for default camera)
cap = cv2.VideoCapture(0)

# Variables for eye closure detection
eye_closed_time = 0
eye_threshold = 0.25  # Time threshold for eye closure (seconds)

# Function to predict if the person is awake or sleepy using the CNN model
def predict_eye_state(frame, x, y, w, h):
    # Crop the region of interest (the face area)
    roi_gray = frame[y:y + h, x:x + w]

    # Detect eyes in the cropped region
    eyes = eye_cascade.detectMultiScale(roi_gray)

    # If no eyes are detected, we consider the eyes as closed
    if len(eyes) == 0:
        return True  # Eyes are closed
    else:
        return False  # Eyes are open

while True:
    # Capture video frame-by-frame
    ret, frame = cap.read()
  # Check if frame is successfully read
    if not ret:
        print("Error: Could not read frame from camera.")
        break  # Exit the loop if frame is not read


    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Check if the eyes are closed for more than the threshold time
        if predict_eye_state(gray, x, y, w, h):
            # If eyes are closed, increment the time the eyes have been closed
            eye_closed_time += 1
        else:
            # If eyes are open, reset the closed time counter
            eye_closed_time = 0

        # Check if eyes have been closed for more than the threshold time (¼ second)
        if eye_closed_time > (eye_threshold * 30):  # Assuming 30 FPS, this is roughly 7-8 frames
            print("ALARM! Eyes Closed!")
            cv2.putText(frame, "ALARM! Eyes Closed!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Play the alarm sound
            playsound(alarm_file_path)  # Play the alarm sound when eyes are closed

    # Display the resulting frame
    cv2.imshow('Eye Tracking and Alarm', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the window
cap.release()
cv2.destroyAllWindows()


Error: Could not read frame from camera.
